In [1]:
import syft as sy
import torch
import torch.nn as nn
from collections import OrderedDict

hook = sy.TorchHook(torch=torch)
bob = sy.VirtualWorker(id="bob")

In [5]:
class MyLayer(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.some_params = nn.Parameter(torch.tensor([5.]))


m = MyLayer()
out = list(m.parameters())
assert len(out) == 1
assert out[0] == m.some_params

In [10]:
ptr = torch.Tensor([1.0, -1.0, 3.0, 4.0]).send(bob)
print(ptr.child)
p = nn.Parameter(ptr)
p.data = ptr
p.data.child = ptr.child
print(p.data.child) # --> trigger attribute error

[PointerTensor - id:44800487555 owner:me loc:bob id@loc:9357626166]
[PointerTensor - id:44800487555 owner:me loc:bob id@loc:9357626166]


In [7]:
p.data.child

[PointerTensor - id:26881423769 owner:me loc:bob id@loc:74911762772]

In [11]:
x = torch.tensor([1,2,3,4.], requires_grad=True).send(bob)
# out_grad = torch.tensor([1.]).send(bob)
# y = x.sum()
# y.backward(out_grad)
# x_grad = bob._objects[x.id_at_location].grad
# x = x.get()
# assert (x.grad == x_grad).all()

In [12]:
x.grad = "adsf"

RuntimeError: expected Variable or None (got str)

tensor(1, dtype=torch.uint8)

In [12]:
x.grad

tensor([1., 1., 1., 1.])

In [46]:
import io

In [47]:
binary_stream = io.BytesIO()
torch.save(x, binary_stream)
tensor_bin = binary_stream.getvalue()

In [49]:
bin_tensor_stream = io.BytesIO(tensor_bin)
tensor = torch.load(bin_tensor_stream)

In [52]:
tensor.grad

In [43]:
x.grad

In [ ]:
x = x.send(bob)
y = (x + x).sum()
grad = torch.tensor([1.]).send(bob)
y.backward(grad)
xgrad = bob._objects[x.id_at_location].grad
xgrad_target = (torch.ones(4).float() + 1)
assert (xgrad == xgrad_target).all()

AssertionError: 

In [7]:
x

(Wrapper)>[PointerTensor - id:29089081352 owner:me loc:bob id@loc:12738504817]

tensor([2., 2., 2., 2.])

In [9]:
x = x.get()

In [11]:
x.grad